In [1]:
# urn and other imports
import unityneuro.render as urn
import numpy as np
import pandas as pd
# ibl imports
from one.api import ONE
one = ONE(base_url='https://alyx.internationalbrainlab.org')
from brainbox.io.one import SpikeSortingLoader
import ibllib.atlas as atlas
ba = atlas.AllenAtlas(25)

C:\Users\Dan\.conda\envs\iblenv\lib\site-packages\one\api.py:1416: UserWarning: Newer cache tables require ONE version 1.13.0 or greater
  warnings.warn(f'Newer cache tables require ONE version {min_version} or greater')


In [5]:
def get_bwm_sessions():

    '''
    Gaelle, 06.04.2022
    '''

    str_query = (  
       'session__project__name__icontains,ibl_neuropixel_brainwide_01,'
       'session__json__IS_MOCK,False,session__qc__lt'
       ',50,~json__qc,CRITICAL,'
       'session__extended_qc__behavior,1,'
       'json__extended_qc__tracing_exists,True,'
       '~session__extended_qc___task_stimOn_goCue_delays__lt,0.9,'
       '~session__extended_qc___task_response_feedback_delays__lt,0.9,'
       '~session__extended_qc___task_wheel_move_before_feedback__lt,0.9,'
       '~session__extended_qc___task_wheel_freeze_during_quiescence__lt,0.9,'
       '~session__extended_qc___task_error_trial_event_sequence__lt,0.9,'
       '~session__extended_qc___task_correct_trial_event_sequence__lt,0.9,'
       '~session__extended_qc___task_reward_volumes__lt,0.9,'
       '~session__extended_qc___task_reward_volume_set__lt,0.9,'
       '~session__extended_qc___task_stimulus_move_before_goCue__lt,0.9,'
       '~session__extended_qc___task_audio_pre_trial__lt,0.9')                  
        
    str_query2 = (
       'session__project__name__icontains,ibl_neuropixel_brainwide_01,'
       'session__json__IS_MOCK,False,session__qc__lt,50,'
       '~json__qc,CRITICAL,session__extended_qc__behavior,1,'       
       'json__extended_qc__tracing_exists,True,'
       'session__extended_qc___experimenter_task,PASS')        
            
    ins = np.concatenate([one.alyx.rest('insertions', 'list', django = x)
                          for x in [str_query, str_query2]])

    eid_probe = set([x['session']+'_'+x['name'] for x in ins]) # pid via x['id']
    ins = [x.split('_') for x in eid_probe] 
 
    return ins

In [29]:
ins = get_bwm_sessions()

In [30]:
ins[0]

['a4000c2f-fa75-4b3e-8f06-a7cf599b87ad', 'probe00']

In [10]:
# def a function to get the pid for an eid/probe combo
def eid2pids(eid):
    insertions = one.alyx.rest('insertions', 'list', session=eid)
    pids = [i['id'] for i in insertions]
    return pids

# Collect insertion data (planned/micro-manip/histology)

In [23]:
def getCoords(ins):
    entry_coords = ba.xyz2ccf(ins.entry)
    tip_coords = ba.xyz2ccf(ins.tip)
    angles = [ins.phi, ins.theta, ins.beta]
    depth = np.sqrt(np.sum(np.power(entry_coords-tip_coords,2)))
    return (entry_coords, tip_coords, angles, depth)

def traj2coords(traj):
    insertion = atlas.Insertion.from_dict(traj)
    return getCoords(insertion)

In [41]:
traj_data = pd.DataFrame(columns=['pid','lab',
                                  'ml_e_p','ap_e_p','dv_e_p',
                                  'ml_t_p','ap_t_p','dv_t_p',
                                  'phi_p','theta_p','depth_p',
                                  'ml_e_m','ap_e_m','dv_e_m',
                                  'ml_t_m','ap_t_m','dv_t_m',
                                  'phi_m','theta_m','depth_m',
                                  'ml_e_e','ap_e_e','dv_e_e',
                                  'ml_t_e','ap_t_e','dv_t_e',
                                  'phi_e','theta_e','depth_e'])

prov_planned = 'planned'
prov_mm = 'Micro-manipulator'
prov_ephys = 'Ephys aligned histology track'

count = 0

for (eid,probe) in ins:
    pids = eid2pids(eid)
    for pid in pids:
        missing = False
        traj_planned =  one.alyx.rest('trajectories', 'list', provenance=prov_planned,probe_insertion=pid,
                              project='ibl_neuropixel_brainwide_01', use_cache=False)
        if len(traj_planned)>0:
            (ecoords_p, tcoords_p, angles_p, depth_p) = traj2coords(traj_planned[0])
        else:
            print(f"{pid} missing planned")
            missing = True

        traj_mm =  one.alyx.rest('trajectories', 'list', provenance=prov_mm,probe_insertion=pid,
                              project='ibl_neuropixel_brainwide_01', use_cache=False)
        if len(traj_mm)>0:
            (ecoords_m, tcoords_m, angles_m, depth_m) = traj2coords(traj_mm[0])
        else:
            print(f"{pid} missing micro-manip")
            missing = True

        traj_ephys =  one.alyx.rest('trajectories', 'list', provenance=prov_ephys,probe_insertion=pid,
                              project='ibl_neuropixel_brainwide_01', use_cache=False)
        if len(traj_ephys)>0:
            (ecoords_e, tcoords_e, angles_e, depth_e) = traj2coords(traj_ephys[0])
        else:
            print(f"{pid} missing ephys")
            missing = True
            
        if missing:
            continue

        traj_data.loc[count] = [pid,traj_planned[0]['session']['lab'],
                            ecoords_p[0],ecoords_p[1],ecoords_p[2],
                            tcoords_p[0],tcoords_p[1],tcoords_p[2],
                            angles_p[0],angles_p[1],depth_p,
                            ecoords_m[0],ecoords_m[1],ecoords_m[2],
                            tcoords_m[0],tcoords_m[1],tcoords_m[2],
                            angles_m[0],angles_m[1],depth_m,
                            ecoords_e[0],ecoords_e[1],ecoords_e[2],
                            tcoords_e[0],tcoords_e[1],tcoords_e[2],
                            angles_e[0],angles_e[1],depth_e]
        count = count+1
    traj_data.to_csv('bwm_all_traj.csv',float_format='%.03f')


4994543f-20e8-4421-adca-8739984e07ad missing planned
30ac1320-d671-46fc-87ef-875cdfc6b4eb missing planned
85bdeae3-269b-4e39-bd9b-2b0d95aff2fa missing planned
117f0d28-3cc0-4837-9e3e-46db5bc3e662 missing planned
54c5cf73-5ea3-47bc-afb5-67ff5a9053a2 missing ephys
0084deda-e6d4-4b47-be23-9e313174995b missing planned
0084deda-e6d4-4b47-be23-9e313174995b missing ephys
85bdeae3-269b-4e39-bd9b-2b0d95aff2fa missing planned
117f0d28-3cc0-4837-9e3e-46db5bc3e662 missing planned
38981973-20de-44e5-8717-efcf88642502 missing planned
d06bdedb-e173-4fe5-a4f1-ab9ab53f9761 missing planned
3283d8ec-1436-4529-8a16-941bbf65c91d missing planned
9338f8bb-e097-46d8-81b4-7f37beb4d308 missing planned
d8c7d3f2-f8e7-451d-bca1-7800f4ef52ed missing planned
692567c3-9982-4ead-a6c3-ecd01a2db2ba missing ephys
ded742e5-23bd-4b5f-8003-58f2eda02bcf missing ephys
6ad5a0a6-362a-4aec-8743-2e91469b0b62 missing planned
b1455ca8-3999-4eb8-9e1e-5c8d0d2e5719 missing planned
c353252d-497a-4dff-8ecd-b14c1dd76f6f missing ephys
97c

In [42]:
traj_data

,pid,lab,ml_e_p,ap_e_p,dv_e_p,ml_t_p,ap_t_p,dv_t_p,phi_p,theta_p,...,depth_m,ml_e_e,ap_e_e,dv_e_e,ml_t_e,ap_t_e,dv_t_e,phi_e,theta_e,depth_e
0,adca9242-0725-41d4-8eb1-5655a464431d,cortexlab,4939.0,4400.0,954.0,5974.276180,4400.000000,4817.703305,180.0,15.0,...,4001.0,4900.000000,4150.000000,1075.0,5535.037085,5073.505888,5252.945974,124.513864,15.016596,4325.663855
1,e8129a86-b5a9-4d2e-9e9c-09689c9bf0b3,cortexlab,4435.0,7650.0,264.0,3285.275416,7650.000000,6784.412133,0.0,10.0,...,6348.0,4471.334154,7498.130724,225.0,3206.935707,7586.750803,6931.193545,4.009232,10.702902,6824.924089
2,4ea45238-55b1-4d54-ba92-efa47feb9f57,cortexlab,3039.0,9900.0,332.0,1778.340982,10010.293373,3808.862697,5.0,20.0,...,3900.0,3175.000000,11200.000000,1475.0,2355.793444,11018.539115,5675.101794,-12.489811,11.297380,4283.092635
3,d4291925-ad00-47fe-baaf-3fdff0991e86,cortexlab,5039.0,8975.0,332.0,5815.457135,8975.000000,3229.777479,180.0,15.0,...,4000.0,5075.000000,8775.000000,275.0,5719.955621,8458.254095,4038.003837,-153.843758,10.810369,3830.991203
4,c6ba6f8e-c13e-410f-b7df-e193ba0d239d,danlab,3471.0,9900.0,692.0,4506.276180,9900.000000,4555.703305,180.0,15.0,...,4000.0,3300.000000,9750.000000,825.0,4018.222006,9886.012241,4584.740640,169.276680,11.002461,3830.142537
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
961,eb99c2c8-e614-4240-931b-169bed23e9f5,danlab,3496.0,6900.0,549.0,4531.276180,6900.000000,4412.703305,180.0,15.0,...,4000.0,2950.000000,7525.000000,625.0,3417.638831,7622.773402,4433.161139,168.190777,7.150651,3838.012112
962,b7612153-372f-415a-9770-7101fbbbdc59,danlab,3484.0,5900.0,740.0,4519.276180,5900.000000,4603.703305,180.0,15.0,...,4000.0,3550.000000,5875.000000,700.0,4731.858110,6262.927320,4275.745886,161.828348,19.181265,3785.925889
963,f4ff3ff8-19b7-40c7-bf6f-8b577b811ed1,churchlandlab,4426.0,5150.0,787.0,3326.459739,5150.000000,7022.802692,0.0,10.0,...,6211.6,4175.000000,5600.000000,650.0,3261.589993,5655.065756,7232.649931,3.449953,7.914092,6645.948404
964,c0c3c95d-43c3-4e30-9ce7-0519d0473911,mrsicflogellab,2309.0,7650.0,978.0,1377.377527,7650.000000,6261.493595,0.0,10.0,...,4911.0,2225.000000,8275.000000,1000.0,1869.698719,9330.211825,5423.600071,71.391062,14.127916,4561.573038


In [ ]:
traj_data = pd.read_csv('bwm_all_traj.csv')

# Discrepancy 1: Planned identical to micro-manipulator
Analyze the *surface* coordinate and check whether it is identical from planned to micro-manipulator, suggesting the researcher did not enter the real micro-manipulator coordinates

In [ ]:
def disc1(row):
    planned = (row['ml_e_p'], row['ap_e_p'], row['dv_e_p'])
    micro = (row['ml_e_m'], row['ap_e_m'], row['dv_e_m'])
    return planned[0]==micro[0] && planned[1]==micro[1] && planned[2]==micro[2]

# Discrepancy 2: >1mm distance from micro-manipulator to ephys
Analyze the surface coordinate to check whether the micro-manipulator coordinate is >1mm away from the ephys coordinate
we'll use only ap/ml and ignore dv for this

In [ ]:
def disc2(row):
    micro = (row['ml_e_m'], row['ap_e_m'], row['dv_e_m'])
    ephys = (row['ml_e_e'], row['ap_e_e'], row['dv_e_e'])
    return np.hypot(micro[0]-ephys[0],micro[1]-ephys[1])

# Discrepancy 3: Above bregma
Check whether the surface coordinate for micro-manipulator or ephys is above bregma (negative values are down, so this 

# Display data using Urchin

In [11]:
import random
r = lambda: random.randint(0,255)
randHexColor = lambda : '#%02X%02X%02X' % (r(),r(),r())

In [22]:
colors = {}
labs = np.unique(traj_data['lab'].values)

for lab in labs:
    colors[lab] = randHexColor()

In [9]:
urn.setup()

(URN) connected to server
Login sent with ID: Dan


In [ ]:
# VIEW 1: Planned insertions
urn.clear()
count = 0

for i, row in traj_data.iterrows():
    probename = 'p'+str(i)
    urn.create_probes([probename])
    urn.set_probe_positions({probename:[row.ml, row.ap, row.dv]})
    urn.set_probe_colors({probename:'#808080'})
    urn.set_probe_angles({probename:[row.phi, row.theta, 0]})
    urn.set_probe_size({probename:[0.07,3.84,0.02]})

# # for some reason 'root' doesn't work?
urn.set_area_visibility({8:True})
urn.set_area_material({8:'transparent-unlit'})
urn.set_area_color({8:'#000000'})
urn.set_area_alpha({8:0.025})

In [56]:
urn.clear()